In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold

In [38]:
npf = pd.read_csv('npf_train.csv')
npf.index=npf['date']
npf['class4'] = npf['class4'].astype('category')
npf.drop(['id', 'partlybad', 'date'], axis=1, inplace=True)
npf.rename(columns={'class4':'class'}, inplace=True)
npf.head()

,class,CO2168.mean,CO2168.std,CO2336.mean,CO2336.std,CO242.mean,CO242.std,CO2504.mean,CO2504.std,Glob.mean,...,T672.mean,T672.std,T84.mean,T84.std,UV_A.mean,UV_A.std,UV_B.mean,UV_B.std,CS.mean,CS.std
date,,,,,,,,,,,,,,,,,,,,,
2000-01-01,nonevent,384.462000,2.284996,384.164462,2.135062,385.274688,2.211695,383.885077,1.955198,19.245511,...,-13.016471,0.525698,-12.422972,0.376324,1.635563,0.856948,0.026438,0.014617,0.003374,0.000733
2000-01-20,nonevent,374.884615,0.415185,374.703333,0.385179,375.621266,0.665720,374.674177,0.435480,31.107659,...,-8.997430,0.373927,-8.351043,0.575679,1.441109,0.741088,0.022649,0.012479,0.001501,0.000572
2000-01-23,nonevent,373.496585,0.189497,373.382593,0.172958,373.961481,0.235107,373.275062,0.165500,29.800885,...,-10.224472,0.965988,-9.651155,1.238891,2.677545,1.261612,0.044759,0.023748,0.000764,0.000048
2000-02-17,nonevent,378.600367,1.934180,378.464862,1.946536,379.785872,2.865022,378.316909,1.983430,23.795211,...,-1.535183,0.122651,-0.829524,0.134191,2.261805,1.345651,0.030893,0.021903,0.002038,0.000751
2000-03-25,Ib,373.128684,1.096617,372.980000,1.047750,373.701830,1.259198,372.910000,1.004164,252.480327,...,-2.095641,1.695622,-1.095864,2.090111,12.906779,7.022300,0.333523,0.239981,0.000662,0.000210


In [41]:
binary = npf.copy()
new_class = np.array(['event']*len(binary), dtype='object')
new_class[binary['class'] == 'nonevent'] = 'nonevent'
binary['class2'] = new_class
binary.drop('class', axis=1, inplace=True)
binary.rename(columns={'class2':'class'}, inplace=True)
binary.head()

,CO2168.mean,CO2168.std,CO2336.mean,CO2336.std,CO242.mean,CO242.std,CO2504.mean,CO2504.std,Glob.mean,Glob.std,...,T672.std,T84.mean,T84.std,UV_A.mean,UV_A.std,UV_B.mean,UV_B.std,CS.mean,CS.std,class
date,,,,,,,,,,,,,,,,,,,,,
2000-01-01,384.462000,2.284996,384.164462,2.135062,385.274688,2.211695,383.885077,1.955198,19.245511,11.909549,...,0.525698,-12.422972,0.376324,1.635563,0.856948,0.026438,0.014617,0.003374,0.000733,nonevent
2000-01-20,374.884615,0.415185,374.703333,0.385179,375.621266,0.665720,374.674177,0.435480,31.107659,24.624718,...,0.373927,-8.351043,0.575679,1.441109,0.741088,0.022649,0.012479,0.001501,0.000572,nonevent
2000-01-23,373.496585,0.189497,373.382593,0.172958,373.961481,0.235107,373.275062,0.165500,29.800885,22.892316,...,0.965988,-9.651155,1.238891,2.677545,1.261612,0.044759,0.023748,0.000764,0.000048,nonevent
2000-02-17,378.600367,1.934180,378.464862,1.946536,379.785872,2.865022,378.316909,1.983430,23.795211,16.178905,...,0.122651,-0.829524,0.134191,2.261805,1.345651,0.030893,0.021903,0.002038,0.000751,nonevent
2000-03-25,373.128684,1.096617,372.980000,1.047750,373.701830,1.259198,372.910000,1.004164,252.480327,138.921953,...,1.695622,-1.095864,2.090111,12.906779,7.022300,0.333523,0.239981,0.000662,0.000210,event


In [70]:
#Multiclass Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB

features = npf.drop('class', axis=1)
target = npf['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
accuracies = list()
for train_index, test_index in kf.split(features):
    features_train = features.iloc[train_index]
    features_test = features.iloc[test_index]
    target_train = target.iloc[train_index]
    target_test = target.iloc[test_index]
    
    model = GaussianNB()
    model.fit(features_train, target_train)
    accuracies.append(model.score(features_test, target_test))
multiclass_GNB = np.array(accuracies).mean()
print(multiclass_GNB)

0.5481159420289854


In [90]:
#Binary Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB

features = binary.drop('class', axis=1)
target = binary['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
accuracies = list()
for train_index, test_index in kf.split(features):
    features_train = features.iloc[train_index]
    features_test = features.iloc[test_index]
    target_train = target.iloc[train_index]
    target_test = target.iloc[test_index]
    
    model = GaussianNB()
    model.fit(features_train, target_train)
    accuracies.append(model.score(features_test, target_test))
binary_GNB = np.array(accuracies).mean()
print(binary_GNB)

0.7665700483091787


In [98]:
#Multiclass Dummy Classifier
from sklearn.dummy import DummyClassifier

features = npf.drop('class', axis=1)
target = npf['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
accuracies = list()
for train_index, test_index in kf.split(features):
    features_train = features.iloc[train_index]
    features_test = features.iloc[test_index]
    target_train = target.iloc[train_index]
    target_test = target.iloc[test_index]
    
    model = DummyClassifier(strategy='most_frequent')
    model.fit(features_train, target_train)
    accuracies.append(model.score(features_test, target_test))
multiclass_dummy = np.array(accuracies).mean()
print(multiclass_dummy)

0.49971014492753624


In [97]:
#Binary Dummy Classifier
from sklearn.dummy import DummyClassifier

features = binary.drop('class', axis=1)
target = binary['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
accuracies = list()
for train_index, test_index in kf.split(features):
    features_train = features.iloc[train_index]
    features_test = features.iloc[test_index]
    target_train = target.iloc[train_index]
    target_test = target.iloc[test_index]
    
    model = DummyClassifier(strategy='most_frequent')
    model.fit(features_train, target_train)
    accuracies.append(model.score(features_test, target_test))
binary_dummy = np.array(accuracies).mean()
print(binary_dummy)

0.45845410628019323


In [201]:
#Multiclass k-Nearest Neighbors Classifier
from sklearn.neighbors import KNeighborsClassifier

features = npf.drop('class', axis=1)
target = npf['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
max_neighbors = 20
multiclass_kNN_acc = list()
for i in range(max_neighbors):
    accuracies = list()
    for train_index, test_index in kf.split(features):
        features_train = features.iloc[train_index]
        features_test = features.iloc[test_index]
        target_train = target.iloc[train_index]
        target_test = target.iloc[test_index]

        model = KNeighborsClassifier(n_neighbors=i+1)
        model.fit(features_train, target_train)
        accuracies.append(model.score(features_test, target_test))
    multiclass_kNN_acc.append(np.array(accuracies).mean())

multiclass_kNN = pd.DataFrame()
multiclass_kNN['Neighbors'] = range(1, max_neighbors+1)
multiclass_kNN['Accuracy'] = multiclass_kNN_acc
multiclass_kNN.index = multiclass_kNN['Neighbors']
multiclass_kNN.drop('Neighbors', axis=1, inplace=True)
multiclass_kNN = multiclass_kNN.sort_values(by='Accuracy', ascending=False)
multiclass_kNN

,Accuracy
Neighbors,
8,0.600531
12,0.594155
7,0.593671
16,0.587053
17,0.585314
4,0.582995
14,0.582802
15,0.580870
10,0.580773


In [200]:
#Binary k-Nearest Neighbors Classifier
from sklearn.neighbors import KNeighborsClassifier

features = binary.drop('class', axis=1)
target = binary['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
max_neighbors = 20
binary_kNN_acc = list()
for i in range(max_neighbors):
    accuracies = list()
    for train_index, test_index in kf.split(features):
        features_train = features.iloc[train_index]
        features_test = features.iloc[test_index]
        target_train = target.iloc[train_index]
        target_test = target.iloc[test_index]

        model = KNeighborsClassifier(n_neighbors=i+1)
        model.fit(features_train, target_train)
        accuracies.append(model.score(features_test, target_test))
    binary_kNN_acc.append(np.array(accuracies).mean())

binary_kNN = pd.DataFrame()
binary_kNN['Neighbors'] = range(1, max_neighbors+1)
binary_kNN['Accuracy'] = binary_kNN_acc
binary_kNN.index = binary_kNN['Neighbors']
binary_kNN.drop('Neighbors', axis=1, inplace=True)
binary_kNN = binary_kNN.sort_values(by='Accuracy', ascending=False)
binary_kNN

,Accuracy
Neighbors,
12,0.796860
9,0.794879
11,0.792802
14,0.792754
19,0.788309
10,0.786184
5,0.785942
18,0.784010
13,0.784010


In [198]:
#Binary Perceptron
from sklearn.linear_model import Perceptron

features = binary.drop('class', axis=1)
target = binary['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
accuracies = list()
for train_index, test_index in kf.split(features):
    features_train = features.iloc[train_index]
    features_test = features.iloc[test_index]
    target_train = target.iloc[train_index]
    target_test = target.iloc[test_index]

    model = Perceptron()
    model.fit(features_train, target_train)
    accuracies.append(model.score(features_test, target_test))

binary_perceptron = np.array(accuracies).mean()
print(binary_perceptron)

0.6752657004830918


In [282]:
#Chained Binary Perceptron
from sklearn.linear_model import Perceptron

k = 10
kf = KFold(n_splits=k, shuffle=True)
accuracies = list()

for train_index, test_index in kf.split(npf):
    train = npf.iloc[train_index]
    test = npf.iloc[test_index]
    
    EvNE_train = train.copy()
    temp = np.array(['event']*len(EvNE_train), dtype='object')
    temp[EvNE_train['class'] == 'nonevent'] = 'nonevent'
    EvNE_train['class'] = temp
    
    IvII_train = train.copy()
    IvII_train = IvII_train[IvII_train['class'] != 'nonevent']
    temp = np.array(['Type I']*len(IvII_train), dtype='object')
    temp[IvII_train['class'] == 'II'] = 'II'
    IvII_train['class'] = temp
    
    AvB_train = train.copy()
    AvB_train = AvB_train[(AvB_train['class'] == 'Ia') | (AvB_train['class'] == 'Ib')]
    temp = np.array(['Ia']*len(AvB_train), dtype='object')
    temp[AvB_train['class'] == 'Ib'] = 'Ib'
    AvB_train['class'] = temp
    
    EvNE_model = Perceptron()
    IvII_model = Perceptron()
    AvB_model = Perceptron()
    EvNE_model.fit(EvNE_train.drop('class', axis=1), EvNE_train['class'])
    IvII_model.fit(IvII_train.drop('class', axis=1), IvII_train['class'])
    AvB_model.fit(AvB_train.drop('class', axis=1), AvB_train['class'])
    
    predictions = list()
    features_test = test.drop('class', axis=1)
    target_test = test['class']
    for date in test.index:
        entry = features_test.loc[date]
        pred = EvNE_model.predict([entry]) #Compare Event vs Nonevent
        if pred == 'nonevent':
            predictions.append(pred[0])
        else:
            pred = IvII_model.predict([entry]) #Compare Type I vs Type II
            if pred == 'Type II':
                predictions.append(pred[0])
            else:
                pred = AvB_model.predict([entry]) #Compare Type Ia vs Type Ib
                predictions.append(pred[0])
    predictions = np.array(predictions)            
    correct_arr = (target_test == predictions).values
    correct = len(predictions[correct_arr])
    accuracies.append(correct/len(predictions))
    
chained_perceptron = np.array(accuracies).mean()
chained_perceptron

0.4322222222222222

In [294]:
#Multiclass Decision Tree
from sklearn.tree import  DecisionTreeClassifier

features = npf.drop('class', axis=1)
target = npf['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
split_acc = list()
for i in range(2, len(features)+1):
    accuracies = list()
    for train_index, test_index in kf.split(features):
        features_train = features.iloc[train_index]
        features_test = features.iloc[test_index]
        target_train = target.iloc[train_index]
        target_test = target.iloc[test_index]

        model = DecisionTreeClassifier(min_samples_split=i)
        model.fit(features_train, target_train)
        accuracies.append(model.score(features_test, target_test))
    multiclass_tree = np.array(accuracies).mean()
    split_acc.append(multiclass_tree)
    
tree_table = pd.DataFrame()
tree_table['Split'] = list(range(2, len(features)+1))
tree_table['Accuracy'] = split_acc
tree_table.sort_values(by='Accuracy', ascending=False, inplace=True)
print(tree_table)

     Split  Accuracy
160    162  0.663285
159    161  0.655217
148    150  0.655024
116    118  0.650725
210    212  0.650386
..     ...       ...
420    422  0.499758
416    418  0.499710
415    417  0.499710
453    455  0.499614
412    414  0.499517

[457 rows x 2 columns]


In [295]:
#Binary Decision Tree
from sklearn.tree import  DecisionTreeClassifier

features = binary.drop('class', axis=1)
target = binary['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
split_acc = list()
for i in range(2, len(features)+1):
    accuracies = list()
    for train_index, test_index in kf.split(features):
        features_train = features.iloc[train_index]
        features_test = features.iloc[test_index]
        target_train = target.iloc[train_index]
        target_test = target.iloc[test_index]

        model = DecisionTreeClassifier(min_samples_split=i)
        model.fit(features_train, target_train)
        accuracies.append(model.score(features_test, target_test))
    binary_tree = np.array(accuracies).mean()
    split_acc.append(binary_tree)
    
binary_tree_table = pd.DataFrame()
binary_tree_table['Split'] = list(range(2, len(features)+1))
binary_tree_table['Accuracy'] = split_acc
binary_tree_table.sort_values(by='Accuracy', ascending=False, inplace=True)
print(binary_tree_table)

     Split  Accuracy
208    210  0.836377
209    211  0.836184
203    205  0.836039
201    203  0.834251
206    208  0.831691
..     ...       ...
428    430  0.428068
455    457  0.421498
427    429  0.412560
417    419  0.412319
437    439  0.410531

[457 rows x 2 columns]


In [310]:
#Multiclass Random Forest
from sklearn.ensemble import RandomForestClassifier

features = npf.drop('class', axis=1)
target = npf['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
entries = list()
for j in range(50, 201, 10): #number of trees
    for i in range(10, 201, 10): #min_split
        accuracies = list()
        for train_index, test_index in kf.split(features):
            features_train = features.iloc[train_index]
            features_test = features.iloc[test_index]
            target_train = target.iloc[train_index]
            target_test = target.iloc[test_index]

            model = RandomForestClassifier(min_samples_split=i, n_estimators=j)
            model.fit(features_train, target_train)
            accuracies.append(model.score(features_test, target_test))
        multiclass_forest = np.array(accuracies).mean()
        entries.append([j, i, multiclass_forest])
        
multiclass_forest_table = pd.DataFrame(data=entries, columns=['# Trees', 'Nodes for Split', 'Accuracy'])
multiclass_forest_table.sort_values(by='Accuracy', ascending=False, inplace=True)
multiclass_forest_table

,# Trees,Nodes for Split,Accuracy
183,140,40,0.672319
240,170,10,0.668213
244,170,50,0.666087
125,110,60,0.666039
264,180,50,0.665942
...,...,...,...
158,120,190,0.571594
299,190,200,0.571401
218,150,190,0.570097
39,60,200,0.569903


In [309]:
#Binary Random Forest
from sklearn.ensemble import RandomForestClassifier

features = binary.drop('class', axis=1)
target = binary['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
entries = list()
for j in range(50, 201, 10): #number of trees
    for i in range(10, 201, 10): #min_split
        accuracies = list()
        for train_index, test_index in kf.split(features):
            features_train = features.iloc[train_index]
            features_test = features.iloc[test_index]
            target_train = target.iloc[train_index]
            target_test = target.iloc[test_index]

            model = RandomForestClassifier(min_samples_split=i, n_estimators=j)
            model.fit(features_train, target_train)
            accuracies.append(model.score(features_test, target_test))
        binary_forest = np.array(accuracies).mean()
        entries.append([j, i, binary_forest])
        
binary_forest_table = pd.DataFrame(data=entries, columns=['# Trees', 'Nodes for Split', 'Accuracy'])
binary_forest_table.sort_values(by='Accuracy', ascending=False, inplace=True)
binary_forest_table

,# Trees,Nodes for Split,Accuracy
160,130,10,0.873382
81,90,20,0.871256
300,200,10,0.871063
102,100,30,0.871063
260,180,10,0.869130
...,...,...,...
118,100,190,0.766039
99,90,200,0.764106
17,50,180,0.764058
179,130,200,0.763961
